In [6]:
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
sys.path.append('../common/')
from utils import *
from optimisation import *
from loo_cv import *
from matplotlib import pyplot as plt 

In [7]:
def compute_error(x_axis,a,b,eps,u_exact,vis=True):
    to_approximate = u_exact
    A =  get_int_matrix(x_axis,eps)
    f = np.array([to_approximate(x) for x in x_axis])
    f=np.append(f,[0.0])
    w = np.linalg.solve(A,f)

    approximator = Interpolator(w,x_axis,eps)

    x_axis_oversampled = np.linspace(a,b,100)
    approximator_oversampled = approximator.evaluate(x_axis_oversampled)
    inter_values = approximator.evaluate(x_axis)
    true_vals = [to_approximate(x) for x in x_axis_oversampled]
    error = np.mean(true_vals-approximator_oversampled)**2
    
    return error, [x_axis, inter_values, x_axis_oversampled, true_vals, approximator_oversampled]

def plot_curves(curves_initial, curves_final, name='test'):
    
    x_axis, inter_values_initial, x_axis_oversampled, true_vals, approximator_initial = curves_initial
    _, _, _, _, approximator_final = curves_final
    
    fig,axs=plt.subplots(ncols=1,nrows=1,figsize=(12,5))
    axs.plot(x_axis_oversampled,true_vals, label = 'True function', color='gray')
    axs.plot(x_axis_oversampled,approximator_initial, label = 'Initial approximation')
    axs.plot(x_axis_oversampled,approximator_final, label = 'Final approximation')
    axs.plot(x_axis,inter_values_initial, linestyle='None', marker='.',label='Interpolation Points',color='green')
    axs.legend()
    
    plt.savefig("epsilon_new/"+name+'.png')
    plt.close()
    
    return None

# Figure 1

In [3]:

for stability in [np.inf]:
    for range_n, eps_v in zip(['interval'],\
                        [np.linspace(0.001,200,2000)]):
        np.random.seed(0)
        for b in [5.0,1.0,0.1,0.01]:
            plt.figure()
            a = 0 
            sample = 1
            n=10
            
            x_axis = sample_from_hypercube(sample,low=a, high=b, n=10)
            x_axis[:,0]=np.sort(x_axis[:,0])
            eps_init = initial_guess(x_axis)
            A =  get_int_matrix(x_axis,eps_init)
            f = lambda x: np.cos(2/b*np.pi*x)+x**2+x
            rhs = np.array([f(x) for x in x_axis])
            
            #eps_v = 
            best_eps = 1.0
            old_error = np.inf
            
            for eps in eps_v:
                error = rippa_cv(eps, x_axis, rhs, stab=stability)
                if error < old_error:
                    best_eps = eps
                    old_error = error
            rippa_eps = best_eps
        
            best_eps = 1.0
            old_error = np.inf
            for eps in eps_v:
                error = mle_cv(eps, x_axis, rhs, stab=stability)
                if error < old_error:
                    best_eps = eps
                    old_error = error
            mle_eps = best_eps
            
            # with optimisation -- the lr probably can be improved
            eps_opt, closs, lr_init_inside, iters = optimization_loop_ADAM(x_axis, eps_init, 0.1)
        
            d = 0
            for i in range(n):
                if i==0:
                    d=x_axis[1]-x_axis[0]
                elif i==n-1:
                    d=d+x_axis[n-1]-x_axis[n-2]
                else:
                    d=d+min(x_axis[i]-x_axis[i-1],x_axis[i+1]-x_axis[i])
            d=d/n
        
            u_exact = f
            
            error, b2 = compute_error(x_axis,a,b,rippa_eps,u_exact,vis=True)
            #print(error)
            
            error, b3 = compute_error(x_axis,a,b,eps_opt,u_exact,vis=True)
            #print(error)
        
            error, b7 = compute_error(x_axis,a,b,mle_eps,u_exact,vis=True)
            
            eps_hardy = 1/(0.815*d)[0]  #Hardy
            eps_franke = 0.8*(10**(1/2))/(2 * (x_axis[n-1]-x_axis[0]))[0]  #Franke
            eps_mfranke = 0.8*(10**(1/4))/(2 * (x_axis[n-1]-x_axis[0]))[0] #Modified Franke
            
            error, b4 = compute_error(x_axis,a,b,eps_hardy,u_exact,vis=True)
            #print(error)
            
            error, b5 = compute_error(x_axis,a,b,eps_franke,u_exact,vis=True)
            #print(error)
            
            error, b6 = compute_error(x_axis,a,b,eps_mfranke,u_exact,vis=True)
            #print(error)
            
            plt.plot(b2[2],b2[4],label=r'Rippa $\varepsilon=$'+f'{rippa_eps:.2f}')
            #plt.plot(b1[2],b1[4],label='loo-cv')
            plt.plot(b4[2],b4[4],label=r'Hardy $\varepsilon=$'+f'{eps_hardy:.2f}')
            plt.plot(b5[2],b5[4],label=r'Franke $\varepsilon=$'+f'{eps_franke:.2f}',linestyle='-.')
            #plt.plot(b6[2],b6[4],label='mfranke')
            plt.plot(b3[2],b3[4],label=r'Optimisation $\varepsilon=$'+f'{np.abs(eps_opt):.2f}')
        
            plt.plot(b7[2],b7[4],label=r'MLE $\varepsilon=$'+f'{mle_eps:.2f}',color='pink',linestyle='dashed')
            
            plt.plot(b3[2],b3[3],label='true',color='black',ls='dotted',lw=2,alpha=0.5)
            
            plt.plot(b3[0],b3[1],ls='None',marker='o',label='pts',color='black')
            plt.legend(loc='best')
            plt.xlim([a,b])
            plt.savefig(f"ranges/range{b}_test_stability_{stability}_{range_n}.pdf")
            plt.close()

/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:50: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:50: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)


update size: 0.045238102464984
loss size: 6.27503944809196


# Appendix: Testing candidate sets

In [4]:
colors = {'A':'blue','B':'red','C':'green'}

for b in [0.1,0.01,0.001]:
        plt.figure()
        for range_n, eps_v in zip(['A','B','C'],\
                [[0.001, 0.002, 0.005, 0.0075, 0.01, 0.02, 0.05, 0.075, 0.1, 0.2, 0.5, 0.75,1,2, 5, 7.5, 10, 20.0, 50, 75, 100, 200, 500, 1000],\
                         np.linspace(0.001,30,200),\
                         np.concatenate([np.linspace(0.001,30,200),[50.0, 75.0, 100.0, 200.0, 500.0, 1000.0]])\
                         ]):
            np.random.seed(42)
            a = 0 
            sample = 1
            n=10
            
            x_axis = sample_from_hypercube(sample,low=a, high=b, n=10)
            x_axis[:,0]=np.sort(x_axis[:,0])
            eps_init = initial_guess(x_axis)
            A =  get_int_matrix(x_axis,eps_init)
            f = lambda x: np.cos(2/b*np.pi*x)+x**2+x
            rhs = np.array([f(x) for x in x_axis])
            
            #eps_v = 
            best_eps = 1.0
            old_error = np.inf
            
            for eps in eps_v:
                error = rippa_cv(eps, x_axis, rhs, stab=np.inf)
                if error < old_error:
                    best_eps = eps
                    old_error = error
            rippa_eps = best_eps
        
            
            error, b2 = compute_error(x_axis,a,b,rippa_eps,u_exact,vis=True)
            plt.plot(b2[2],b2[4],label=r'Rippa $\varepsilon=$'+f'{rippa_eps:.2f} Set: {range_n}',color=colors[range_n])
            
        plt.plot(b2[2],b2[3],label='true',color='black',ls='dotted',lw=2,alpha=0.25)
        plt.plot(b2[0],b2[1],ls='None',marker='o',label='pts',color='black')
        plt.legend(loc='best')
        plt.xlim([a,b])
        plt.savefig(f"NEW_range{b}_test_stability_rippa.pdf")
        plt.close()

/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:50: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:50: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)
/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/1-dimensional/../common/loo_cv.py:50: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)


# Appendix: candidate set Rippa

/tmp/slurmtmp.659415/ipykernel_3047261/822580715.py:41: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)
/tmp/slurmtmp.659415/ipykernel_3047261/822580715.py:42: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)
/tmp/slurmtmp.659415/ipykernel_3047261/822580715.py:19: RuntimeWarning: divide by zero encountered in divide
  error_vector = np.divide(w,np.diagonal(invM))
/tmp/slurmtmp.659415/ipykernel_3047261/822580715.py:41: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)
/tmp/slurmtmp.659415/ipykernel_3047261/822580715.py:42: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)
/tmp/slurmtmp.659415/ipykernel_3047261/822580715.py:19: RuntimeWarning: divide by zero encountered in divide
  error_vector = np.divide(w,np.diagonal(invM))
/tmp/slurmtmp.659415/ipykernel_3047261/822580715.py:41: RuntimeWarning: invalid value encountered in log
 